# Statistical Modeling

In [ ]:
import os
notebook_dir = os.getcwd()
parent_path=os.path.dirname(notebook_dir)

os.chdir(parent_path)

In [ ]:
from scripts.data_processing import DataProcessing
from scripts.model import Modelling

In [ ]:
import pandas as pd
insurance_df = pd.read_csv('data\\insurance_data.csv',low_memory=False)

In [ ]:
# instanciate
processor=DataProcessing()
model=Modelling()